## Youtube to MP3

### Youtube DL

In [2]:
!pip install youtube_dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 14.6 MB/s 


In [3]:
import youtube_dl

In [38]:
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=BfKv8xeDv_Y']) #Gros Temps de chargement

[youtube] BfKv8xeDv_Y: Downloading webpage
[download] Destination: États-Unis  - un Noël à -48°-BfKv8xeDv_Y.m4a
[download]  26.2% of 1.47MiB at 46.18KiB/s ETA 00:24

KeyboardInterrupt: ignored

### Pytube

In [1]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 1.9 MB/s 


In [2]:
from pytube import YouTube as yt

In [3]:
link = "https://www.youtube.com/watch?v=qkLRozfca50"

In [4]:
audio = yt(link).streams.filter(only_audio=True).all()

<ipython-input-4-ce8c393b1174>:1: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  audio = yt(link).streams.filter(only_audio=True).all()


In [5]:
filename = audio[0].download("/content/mp3")
filename

'/content/mp3/Coding An Automated YouTube Channel!.mp4'

In [6]:
print(filename)

/content/mp3/Coding An Automated YouTube Channel!.mp4


## MP3 to Text

In [8]:
!pip install git+https://github.com/openai/whisper.git -q

     |████████████████████████████████| 5.8 MB 27.2 MB/s 
     |████████████████████████████████| 7.6 MB 50.2 MB/s 
     |████████████████████████████████| 182 kB 56.2 MB/s 


In [9]:
import whisper

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


### Large

In [7]:
whisp = whisper.load_model('large') # Gros Temps de chargement

100%|█████████████████████████████████████| 2.87G/2.87G [00:54<00:00, 56.6MiB/s]


In [8]:
Text = whisp.transcribe(new_name,  fp16=False) # Gros Temps de chargement

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: ignored

### Base

In [10]:
WhispBase = whisper.load_model('base')

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 147MiB/s]


In [11]:
%%time
Text = WhispBase.transcribe(filename,  fp16=False)

CPU times: user 23.1 s, sys: 940 ms, total: 24.1 s
Wall time: 32.4 s


In [12]:
Text_to_summarize = Text['text']

In [13]:
Text_to_summarize

" Alright, so today we're building an automated YouTube channel. Alright, so this is going to require a little bit of a couple of things which is first we need to do build some sort of web scraper that will check reddit is what I'm thinking and basically just find interesting stories that we can turn from text into speech and then into an audio file that we can then convert into some sort of video and then that video is what we'll get uploaded to YouTube. So we need to automate that entire process like finding a story, downloading it, converting it to an audio file and then into a video file and then using the YouTube API to actually upload it to YouTube. So the first thing that we're going to do is figure out how to web scrape reddit and find some place where we can actually find interesting stories that we can turn from text to audio to video. Alright, so so far what I've done is I've just created a list of some of the to-do's and I've done this on GitHub at the project boards. So if

## Summarization

In [14]:
import torch
from transformers import pipeline

In [15]:
summarizer = pipeline(
    "summarization",
    "pszemraj/long-t5-tglobal-base-16384-book-summary",
    device=0 if torch.cuda.is_available() else -1,
) #maybe change size of the model

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [16]:
params = {
    "max_length": 1000,
    "min_length": 258,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.3,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 4,
} # parameters for text generation out of model

In [17]:
%%time
result = summarizer(Text_to_summarize, **params) #gros temps de chargement 

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:722: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


CPU times: user 55.9 s, sys: 303 ms, total: 56.2 s
Wall time: 57 s


In [18]:
summary = result[0]["summary_text"]

In [19]:
summary

'In this chapter, the narrator tells us that he\'s building an "automated" YouTube channel to collect stories and convert them into something useful. He wants to make it easy for people to share these stories with other people so they can watch them in their own time. He also talks about how he has been using a bunch of different tools to help him get things done. For example, he uses a tool called Pip to convert his voice into mp3, which is then used to create another kind of video. The next thing he does is find some stories that are interesting enough to be converted into little pieces of music. This will then be uploaded to the big-time streaming site, Twitch. Unfortunately, though, there\'s no way he can actually do this without getting approval from the authorities. So here\'s what he plans to do: he decides to build an entire stream of content that can be watched by anyone who wants to see it. If you want to know more details, check out our guide on how to set up your own custom

## Text to PDF

In [20]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 21.6 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=04a42e08bc3b503045d34f21c75589ff00660f04f3f1eca378a86071c39b623d
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [22]:
from docx import Document

document = Document()
document.add_heading('Summary', level=1)
document.add_paragraph(summary)

document.save('/content/pdf/docx_file.docx')

In [23]:
!pip install aspose-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68.8 MB 33 kB/s 


In [24]:
import aspose.words as aw

doc = aw.Document("/content/pdf/docx_file.docx")
doc.save("/content/pdf/summary2.pdf")